In [2]:
import pandas as pd
import os
import csv
import sys
import numpy as np

csv.field_size_limit(100000000)

131072

## Processing https://www.kaggle.com/jruvika/fake-news-detection Dataset into Combined Data

In [4]:
from urllib.parse import urlparse

jruvika_csv = pd.read_csv('./work/Datasets/Kaggle/Dataset_1/data.csv', engine='python')
jruvika_csv.columns = ['url', 'title', 'content', 'label']
jruvika_csv['label'] = np.where(jruvika_csv['label'] == 1, 'reliable', 'fake')

# Extracting out domain from URL
def extract_domain(url):
    partialDomain = urlparse(url).netloc
    return partialDomain.replace("www.", "")
jruvika_csv['domain'] = jruvika_csv['url'].apply(lambda x: extract_domain(x))

jruvika_csv

# Saving to CSV
jruvika_csv.to_csv('./work/Datasets/Combined_Dataset/combined_data.csv', mode='a', index=False)

print('Done')

Done


## Processing LIAR dataset
### Some references to: https://towardsdatascience.com/identifying-fake-news-the-liar-dataset-713eca8af6ac

In [5]:
liar_csv = pd.read_table('./work/Datasets/liar_dataset/train.tsv', names = ['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
                                            'barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire', 'venue'])

liar_csv = liar_csv.drop(columns=['id', 'subject', 'job', 'state', 'party', 'barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire', 'venue'])

liar_label_dict = {
    "pants-fire" : 'fake', 
    "false" : 'fake', 
    "barely-true" : 'fake', 
    "half-true" : 'fake', 
    "mostly-true" : 'reliable', 
    "true" : 'reliable'
}
liar_csv['label'] = liar_csv['label'].apply(lambda x: liar_label_dict[x])


liar_csv.to_csv('./work/Datasets/Combined_Dataset/truncated_liar_dataset.csv', mode='a', index=False)
print("Done")

Done


## Processing Fakenewscorpus Datas into Combined Data

In [3]:
fakenewscorpus_csv_iter = pd.read_csv('./work/Datasets/FakeNewsCorpus/news_cleaned_2018_02_13.csv', engine='python', iterator=True, chunksize=100000, encoding='utf-8', header=0, error_bad_lines=False)

label_dict = {
    'political': 'reliable',
    'bias': 'fake',
    'unreliable': 'fake',
    'reliable': 'reliable',
    'fake': 'fake'
}

for single_df in fakenewscorpus_csv_iter:
    single_df = single_df[(single_df.type == 'fake') | (single_df.type == 'reliable') | (single_df.type == 'political') | (single_df.type == 'bias') | (single_df.type == 'unreliable')]

    single_df.dropna()
    single_df = single_df.rename({'type': 'label', 'authors': 'author'}, axis='columns')
    single_df = single_df.drop(columns=['id', 'scraped_at', 'inserted_at', 'updated_at', 'keywords', 'meta_keywords', 'meta_description', 'tags', 'summary'])


    single_df['label'] = single_df['label'].apply(lambda x: label_dict[x])
    single_df.to_csv('./work/Datasets/Combined_Dataset/truncated_corpus.csv', mode='a')

print("FINSIHED WITH CORPUS DATA")

FINSIHED WITH CORPUS DATA
